# Naive Bayes
Вам необходимо реализовать наивный байесовский классификатор для анализа тональности твитов. Анализ тональности, в нашем случае, это простая задача бинарной классификации, в которой вам предстоит определить явяляется ли твит позитивным (по настроению) или негативным.

Этапы решения задачи:

* Обучить модель наивного байесовского классификатора для определения тональности текста.
* Протестировать модель на валидационном наборе данных.
* Вычислить отношение позитивных слов к негативным.
* Проанализировать ошибки.
* Протестировать модель на новых твитах.

В данном задании приведены некоторые шаблоны функций, которые вам нужно реализовать, однако, если вам не нравится их нотация, либо вы хотите выполнить задание по-другому, можете не использовать эти шаблоны.

```nltk``` - это библиотека, которая содержит ряд инструментов по работе с текстовыми данными. 

Например, с помощью нее можно загружать стандартные датасеты (например, тексты твитов для классификации), список стоп-слов.

In [312]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.sustech.edu.cn/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [313]:
# загружаем необходимые библиотеки
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from os import getcwd

# Загружаем стандартный набор твитов из библиотеки nltk 
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [314]:
import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import numpy as np 

In [315]:
getcwd()

'c:\\Users\\Leo\\Desktop\\Misis\\ML_1_sem\\Project_3'

In [316]:
# Получаем доступ к директории, в которой хранятся твиты
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [317]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

# стандартный сплит данных на test и train
# можете реализовать с помощью train_test_split, если посчитаете нужным
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

# приписываем соответствующие метки классов
# если делали деление с помощью train_test_split, лучше добавить метки классов сразу к исходным данным
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

# Часть 1: Предобработка данных

Итак, для построения наивного байесовского классификатора каждый текст должен быть описан списком релевантных токенов.

Токены - это смысловая часть слова, которая может часто встречатться в языке. Для того чтобы выделить токены из текста воспользуемся классом ```TweetTokenizer``` библиотеки nltk.

In [318]:
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    
input_text = "I like Machine Learning"
text = tokenizer.tokenize(input_text)
print(text)

['i', 'like', 'machine', 'learning']


Однако чтобы находить сходство между текстами, необходимо, чтобы разные формы слов (время, падеж, суффиксы, приставки) воспринимались как один токен, например, *like* и *likes*.

Для этого нужно воспользоваться стеммингом. Стемминг - процесс приведения слова к его стемме (по сути, к корню).

В библиотеке ```nltk``` реализован класс [```PorterStemmer```](https://www.nltk.org/howto/stem.html).

In [319]:
stemmer = PorterStemmer()
print(stemmer.stem('likes'))
print(stemmer.stem('apple'), stemmer.stem('apples'))

like
appl appl


Также при обработке текстовых данных часто требуется удалять лишние символы. В некоторых задачах это могут быть знаки препинания, служебные символы, гиперссылки и т.д. Они не несут в себе семантической нагрузки и могут помешать классификатору в определении класса текста.

Удаление лишних символов можно производить с помощью регулярных выражений модуля ```re```.

Например, чтобы удалить знаки пунктуации из строки, можно выполнить следующую функцию:

In [320]:
text = "Hello! What a nice day!!!"
text = re.sub(r'[.,"\'-?:!;]', '', text)
print(text)

Hello What a nice day


### Задание 1.

Проведите предобработку данных:
- Удаление шума: удаляем слова, которые не имеют значения для решаемой задачи. Предположительно, это должны быть такие общие слова как 'I, you, are, is, etc...'.
- Также необходио удалить служебные символы, гиперссылки, хэштеги и т.д.
- Знаки пунктуации (опционально), во-первых стоит удалить знаки пунктуации, если они нарушают выделение уникального слова, например, "happy", "happy?", "happy!", "happy,", значимое слово здесь "happy", нам необходимо выделить его как единый токен. Однако, вы можете попробовать выделить в осмысленные токены значимую комбинацию знаков пунктуации, например, ':-)', ')', '(' и т.д.
- Произвести стемминг, привести слова к нормальной форме. Например, такие слова как "motivation", "motivated", and "motivate" будут представлены одной формой "motiv".

Реализуйте функцию `process_tweet` для проведения предобработки твитов.

In [321]:
def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    tweet_new = ''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    
    # удаления символов типа $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    
    # удалени символа ретвита "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # удалите гиперссылки
    tweet = re.sub(r"https?://[^,\s]+,?", "", tweet)
    # поиск смайлика
    smiley = re.findall(r'(?:\<3|\(?\^\^\)?|[-^]?[@:;][oO]?[D\)\]\(\]/\\OpP])', tweet)

    # удалите символ '#' для того, чтобы работать с хэштегами как с обычными словами
    tweet = re.sub(r'\#', '', tweet)
    tweet = re.sub(r"@\S+", '', tweet)
    tweet =re.sub(r'[^a-zA-Z0-9\s^|]', '', tweet) 
    tweet = re.sub(r'[.,"\'-?:!;]', '', tweet)
    
    # токенизация твитов
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    
    tweet_tokens = tokenizer.tokenize(tweet)

    temp_list = [stemmer.stem(word) for word in tweet_tokens if word.lower() not in stopwords_english]
    if len(smiley) >0:
        for i in smiley:
            temp_list.append(i)
    tweets_clean = []
    # токенизируйте текст и приведите слова к нормальной форме
    
    for word in temp_list:
         tweets_clean.append(word)

    return tweets_clean



In [322]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# проверьте функцию для предобработки твитов
print(process_tweet(custom_tweet))

['hello', 'great', 'day', 'good', 'morn', ':)']


## Часть 1.1 Реализация вспомогательных функций

Чтобы рассчитать $P(w_i|class)$ необходимо усеть рассчитывать частоту $(w_i|class)$ для всех текстов из обучающей выборки.

Необходимо завести словарь, где ключом будет являться кортеж (слово, класс), а значения, соответствующие этому кортежу, частотой в обучающем корпусе. Класс 1 указывает позитивную тональность, 0 - на негативную.

Необходимо реализовать вспомогательную функцию поиска, которая принимает словарь `freqs`, слово и класс (1 или 0) и возвращает количество раз, которое кортеж (слово, класс) встречались в корпусе.

Например, для списка твитов `["i am rather excited", "you are rather happy"]`, соответствующих классу 1, функция вернет словарь:

{
    ("rather", 1): 2,
    ("happi", 1) : 1, 
    ("excit", 1) : 1
}

#### Задания
Реализуйте функцию `count_tweets`, которая получает на вход список твитов, производит их предобработку и взвращает соответствующий словарь.
- Ключом словаря является кортеж, содержащий нормальную форму слова и метку класса word, например, ("happi",1).
- Значение - сколько раз это слово появилось в текстах соответствующих классов (integer).

In [323]:
def count_tweets(tweets, ys):
    '''
    Input:
        tweets: список твитов
        ys: класс твита (0 или 1)
    Output:
        result: словарь, сопоставляющий пары (слово, класс) частоте
    '''
    result = {}
    map_count = {}
    for i, words in enumerate(tweets):
        temp_list = process_tweet(words)
        for word in temp_list:
            result_tuple = (word, ys[i])
            # Добавляем новый ключ в словарь со значением 1,
            # если слово не было использовано с этим классом
            if result.get(result_tuple, None) is None:
                result[(word, ys[i])] = 1
            else:
                result[(word, ys[i])] += 1
    return result


In [324]:
# Тестирование функции

tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
result = count_tweets(tweets, ys)
result

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

**Ожидаемый результат**: {('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

# Часть 2: Обучите наивный байесовский классификатор

Наивный Байес - это алгоритм, который можно использовать для анализа тональности текстов. Алгоритм быстрый, не требует много времени для обучения.

#### Процесс обучения наивного байесовского классификатора
- Определить число классов.
- Рассчитать вероятность для каждого класса.
$P(D_{pos})$ - вероятность увидеть документ с положительной тональностью.
$P(D_{отрицание})$ - вероятность увидеть документ с отрицательной тональностью.

$$P(D_{pos}) = \frac{D_{pos}}{D}\tag{1}$$

$$P(D_{neg}) = \frac{D_{neg}}{D}\tag{2}$$

где $D$ - общее количество документов, или твитов в данном случае, $D_{pos}$ - общее количество положительных твитов, а $D_{neg}$ - общее количество отрицательных твитов.

#### Prior and Logprior

Априорная вероятность (Prior) - вероятность того, что мы можем получит позитивный твит в нашем корусе документов. Prior определяется как отношение вероятностей $\frac{P(D_{pos})}{P(D_{neg})}$.

Для удобства можем рассчитать логарифм этой вероятности - logprior:

$\text{logprior} = log \left( \frac{P(D_{pos})}{P(D_{neg})} \right) = log \left( \frac{D_{pos}}{D_{neg}} \right)$.

Так как $log(\frac{A}{B}) = log(A) - log(B)$:

$\text{logprior} = \log (P(D_{pos})) - \log (P(D_{neg})) = \log (D_{pos}) - \log (D_{neg})\tag{3}$.



#### Положительная и отрицательная вероятность слова

Чтобы вычислить положительную вероятность и отрицательную вероятность для определенного слова в словаре, мы будем использовать следующие входные данные:

- $freq_{pos}$ и $freq_{neg}$ являются частотами этого конкретного слова в положительном или отрицательном классе. Другими словами, положительная частота слова - это количество раз, когда слово подсчитывается с меткой 1.
- $N_{pos}$ и $N_{neg}$ - общее количество положительных и отрицательных слов для всех документов (для всех твитов) соответственно.
- $V$ - это количество уникальных слов во всем наборе документов для всех классов, как положительных, так и отрицательных.

Мы будем использовать их для вычисления положительной и отрицательной вероятности для определенного слова, используя эту формулу:

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

Здесь реализуется сглаживание Лапласа. [Здесь](https://en.wikipedia.org/wiki/Additive_smoothing) можно прочитать про процедуру сглаживания.

#### Log likelihood
Для вычисления loglikelihood можно воспользоваться следующей формулой:

$$\text{loglikelihood} = \log \left(\frac{P(W_{pos})}{P(W_{neg})} \right)\tag{6}$$

##### Теперь нужно создать словарь частот (`freqs`)
- Применяя функцию `count_tweets`, можно вычислить словарь `freqs`, который содержит все частоты.
- В словаре `freqs` ключом является кортеж (слово, *класс*).
- Значение - частота появления слова в текстах соответствующих *классу*.

In [325]:
# Построение словаря частот
freqs = count_tweets(train_x, train_y)
len(freqs)

10733

#### Задания

Дано: словарь частот, `train_x` (список твитов) и `train_y` (список классов, характеризующих тональность твита). Необходимо реализовать наивный байесовский классификатор.

##### Вычисление $V$
- Необходимо рассчитать количество уникальных слов в словаре `freqs`.

##### Вычисление $freq_{pos}$ и $freq_{neg}$
- Используя словарь `freqs`, можно вычислить частоты для позитивных и негативных элементов: $freq_{pos}$ и $freq_{neg}$.

##### Вычисление $N_{pos}$ и $N_{neg}$
- Используя словарь `freqs`, можно также вычислить общее количество слов в позитивных и негативных документах: $N_{pos}$ и $N_{neg}$.

##### Вычисление $D$, $D_{pos}$, $D_{neg}$
- Используя входной список классов `train_y`, вычислите общее количество документов $D$, а также количество позитивных и негативных документов: $D_{pos}$ и $D_{neg}$.
- Вычислите вероятность того, что твит имеет положительную тональность $P(D_{pos})$, а также вероятность, что твит имеет отрицательную тональность $P(D_{neg})$.

##### Вычислите logprior
- logprior = $log(D_{pos}) - log(D_{neg})$

##### Вычислите log likelihood
- В заключение нужно пройти по всем словам в словаре, для этого нужно использовать функцию `lookup` для получения положительных и отрицательных частот: $freq_{pos}$ и $freq_{neg}$ для каждого слова.
- Вычислите $P(W_{pos})$ и $P(W_{neg})$ используя формулы 4 и 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

- Далее можно вычислить loglikelihood: $log \left( \frac{P(W_{pos})}{P(W_{neg})} \right)$.

In [326]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: словарь с частотами для каждой пары
        word: слово, которое нужно найти
        label: класс, соответствующий слову
    Output:
        n: количество раз, которое слово и соответствующая ему метка появились в корпусе.
    '''
    n = 0 

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

def test_lookup(func):
    freqs = {('sad', 0): 4,
             ('happy', 1): 12,
             ('oppressed', 0): 7}
    word = 'happy'
    label = 1
    if func(freqs, word, label) == 12:
        return 'SUCCESS!!'
    return 'Failed Sanity Check!'

In [327]:
import math
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: словарь: {(слово, класс), частота}
        train_x: список твитов
        train_y: список классов (0,1), соответствующих твитам
    Output:
        logprior: log prior (формула 3).
        loglikelihood: log likelihood (формула 6).
    '''
    loglikelihood = {}
    logprior = 0

    # подсчет уникальных значений
    # подсчет n_pos and n_neg
    uniq = set()# уникальные значения слов
    uniq_pos = set() 
    uniq_neg = set()
    for word in freqs:
        if word[1]==1:
            uniq_pos.add(word[0])
        else:
            uniq_neg.add(word[0])
        uniq.add(word[0])
    n_pos = len(uniq_pos)
    n_neg = len(uniq_neg)
    uniq_count = len(uniq) # кол-во уникальных значений

    # подсчет d, d_neg, d_pos
    d = len(train_y)
    d_pos = len(train_y[train_y == 1])
    d_neg = len(train_y[train_y == 0])

    # подсчет p_d_neg, p_d_pos
    p_d_neg = d_neg / d 
    p_d_pos = d_pos / d


    # подсчет logprior
    logpror = math.log(d_pos) - math.log(d_neg)

    # подсчет log likelihood
    for word in uniq:
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)
        p_w_pos = (freq_pos + 1) / (n_pos + uniq_count)
        p_w_neg = (freq_neg + 1) / (n_neg + uniq_count)
        loglikelihood[word] = math.log(p_w_pos) - math.log(p_w_neg)

    return logprior, loglikelihood



In [328]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0
8563


**Ожидаемый результат**:

0.0

91.65

# Часть 3: Тестирование классификатора

Сделаем прогноз на тестовых твитах.

#### Реализовать функцию `naive_bayes_predict`
Задания:

Реализуйте функцию `naive_bayes_predict`, чтобы делать прогнозы в твитах.

* Функция принимает `tweet`, `logprior`, `loglikelihood`.
* Функция возвращает вероятность того, что твит относится к позитивному или негативному классу.

$$ p = logprior + \sum_i^N (loglikelihood_i)$$

#### Примечание
Обратите внимание, сбалансирован ли набор данных, с которым вы работаете в этом задании, и чему будет равен logprior.

In [329]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: строка
        logprior: число
        loglikelihood: словарь, переводящий слово в соответствующий ему logprior
    Output:
        p: сумма всех logliklihoods для каждого слова из твита (если оно содержится в словаре) + logprior (число)

    '''
    word_l = 0
    words = process_tweet(tweet)
    p = logprior + sum(loglikelihood.get(stem, 0) for stem in words)
    return p

In [330]:
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.5367220996043542


**Ожидаемый результат**:
- Ожидаемый результат - около 1.55
- Тональность позитивная.

#### Реализуйте функцию test_naive_bayes
**Описание функции**:
* Функция `test_naive_bayes` оценивает точность предсказаний.
* Функция принимает на вход `test_x`, `test_y`, log_prior и loglikelihood
* Функция возвращает точность для тестовой выборки.

In [331]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    """
    Input:
        test_x: список твитов
        test_y: соответствующие классы для твитов
        logprior: logprior
        loglikelihood: словарь с loglikelihoods для каждого слова
    Output:
        accuracy: точность предсказания
    """
    accuracy = 0 

    pred_y = np.array(
        [naive_bayes_predict(tweet, logprior, loglikelihood) > 0 for tweet in test_x]
    )

    error = np.abs(test_y - pred_y).mean()
    accuracy = 1 - error

    return accuracy

In [332]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9595


Не понимаю как достичь такой точности(

**Ожидаемая точность**:

`Точность наивного байесовского классификатора = 0.9955`

In [333]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:    
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
    print(f'{tweet} -> {p:.2f}')

I am happy -> 2.13
I am bad -> -1.19
this movie should have been great. -> 2.06
great -> 2.13
great great -> 4.26
great great great -> 6.39
great great great great -> 8.52


**Ожидаемый результат**:
- I am happy -> 2.14
- I am bad -> -1.31
- this movie should have been great. -> 2.12
- great -> 2.13
- great great -> 4.26
- great great great -> 6.39
- great great great great -> 8.52

In [334]:
# Проверьте тональность на любом твите
my_tweet = 'you are bad :('
naive_bayes_predict(my_tweet, logprior, loglikelihood)

-8.734857373068746

# Часть 4: Отфильтруйте слова по соотношению положительных и отрицательных значений

- Некоторые слова чаще втречаются в положительном контексте, чем другие, и их можно считать "более позитивными". Аналогично, другие слова можно считать "более негативными".
- Один из способов определить уровень позитивности или негативности, не вычисляя логарифмическую вероятность, - сравнить положительную и отрицательную частоту слова.
- Можно использовать расчеты логарифмического правдоподобия (log likelihood) для сравнения относительной позитивности или негативности слов.
- Для каждого слова можно рассчитать соотношение положительных и отрицательных частот этого слова.
- Слова можно отфильтровать в соответствии с рассчитанными соотношениями.

#### Реализовать функцию get_ratio

$$ ratio = \frac{\text{pos_words} + 1}{\text{neg_words} + 1} $$

где pos_words и neg_words сответствуют частотам слов по классам. 
<table>
    <tr>
        <td>
            <b>Words</b>
        </td>
        <td>
        Positive word count
        </td>
         <td>
        Negative Word Count
        </td>
  </tr>
    <tr>
        <td>
        glad
        </td>
         <td>
        41
        </td>
    <td>
        2
        </td>
  </tr>
    <tr>
        <td>
        arriv
        </td>
         <td>
        57
        </td>
    <td>
        4
        </td>
  </tr>
    <tr>
        <td>
        :(
        </td>
         <td>
        1
        </td>
    <td>
        3663
        </td>
  </tr>
    <tr>
        <td>
        :-(
        </td>
         <td>
        0
        </td>
    <td>
        378
        </td>
  </tr>
</table>

In [335]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: словарь, соодержащий слова

    Output: словарь с ключами: 'positive', 'negative' и 'ratio'.
        Пример: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    pos_neg_ratio["positive"] = freqs.get((word, 1), 0)
    pos_neg_ratio["negative"] = freqs.get((word, 0), 0)
    pos_neg_ratio["ratio"] = (pos_neg_ratio["positive"] + 1) / (pos_neg_ratio["negative"] + 1)
   
    return pos_neg_ratio


In [336]:
get_ratio(freqs, 'happi')

{'positive': 161, 'negative': 18, 'ratio': 8.526315789473685}

#### Реализуйте функцию get_words_by_threshold(freqs,label,threshold)

* Используйте функцию `get_ratio` для составления словаря, состоящего из 'положительного' и 'отрицательного' количества, а также ratio для этих значений.
* Добавьте составленный словарь `get_ratio` в качестве значения для другого словаря, в котором ключом является само слово.

Пример элемента словаря:
```
{'happi':
    {'positive': 10, 'negative': 20, 'ratio': 0.524}
}
```

In [337]:
def get_words_by_threshold(freqs, label, threshold, get_ratio=get_ratio):
    '''
    Input:
        freqs: словарь слов
        label: 1, 0
        threshold: ratio - порог, по которому мы будем решать, включать ли слово в словарь
    Output:
        word_list: словарь, содержащий слово и информацию о его "позитивном количестве", "негативном количестве", и отношении (ratio) "позитивного количества" к негативному.
        Пример:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}
    for word in freqs:
        pos_neg_ratio = get_ratio(freqs, word[0])
        if ((label == 1) and (pos_neg_ratio["ratio"] >= threshold)) or (
            (label == 0) and (pos_neg_ratio["ratio"] <= threshold)):
            word_list[word[0]] = pos_neg_ratio
    
    return word_list



In [338]:
get_words_by_threshold(freqs, label=0, threshold=0.05)

{':(': {'positive': 1, 'negative': 3723, 'ratio': 0.0005370569280343716},
 'zayniscomingbackonjuli': {'positive': 0, 'negative': 19, 'ratio': 0.05},
 'belev': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'wll': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776},
 'justn': {'positive': 0, 'negative': 35, 'ratio': 0.027777777777777776}}

In [339]:
get_words_by_threshold(freqs, label=1, threshold=10)

{'followfriday': {'positive': 23, 'negative': 0, 'ratio': 24.0},
 'commun': {'positive': 27, 'negative': 1, 'ratio': 14.0},
 ':)': {'positive': 2966, 'negative': 4, 'ratio': 593.4},
 'flipkartfashionfriday': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 ':D': {'positive': 529, 'negative': 0, 'ratio': 530.0},
 'p': {'positive': 107, 'negative': 3, 'ratio': 27.0},
 ':p': {'positive': 104, 'negative': 0, 'ratio': 105.0},
 'influenc': {'positive': 16, 'negative': 0, 'ratio': 17.0},
 'here': {'positive': 20, 'negative': 0, 'ratio': 21.0},
 'youth': {'positive': 15, 'negative': 0, 'ratio': 16.0},
 'bam': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'warsaw': {'positive': 44, 'negative': 0, 'ratio': 45.0},
 'shout': {'positive': 11, 'negative': 0, 'ratio': 12.0},
 ';)': {'positive': 22, 'negative': 0, 'ratio': 23.0},
 'stat': {'positive': 51, 'negative': 0, 'ratio': 52.0},
 'arriv': {'positive': 57, 'negative': 4, 'ratio': 11.6},
 'glad': {'positive': 41, 'negative': 2, 'ratio': 14.0},

Проанализируйте, насколько интуитивно были определены "позитивные" и "негативные" слова.

# Часть 5: [Бонус] Анализ ошибок

Проанализируйте твиты, которые были неправильно классифицированы вашей моделью. Как вы считаете, с чем связаны ошибки?

In [340]:
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    # you code here

Truth Predicted Tweet


# Часть 6: Предскажите тональность какого-нибудь интересного твита

In [341]:
# Test with your own tweet - feel free to modify `my_tweet`
my_tweet = 'I am happy because I am learning :)'

p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

9.056120624523112


# Приведите краткий отчет по выполненному заданию

В данной работе я ознакомился с библиотекой nltk. Понял работу наивного байесовского классификатора